# Fetching locations

In [ ]:
SELECT name, nameID as locationID, id as _id
FROM [iot].[locations];

# Fetching data

In [ ]:
-- per minute
DECLARE @location VARCHAR(50);
SET @location = 'churchill-1c';

SELECT TOP 1000 dateadd(mi, datediff(mi, 0, timestamp), 0) as timestamp, AVG(temperatureDHT) as temperature, AVG(pressureLPS) as pressure, AVG(humidityDHT) as humidity, AVG(eco2) as eco2, AVG(wifiDevices) as wifiDevices, COUNT(timestamp) as '_count'
FROM [iot].[messages] m
INNER JOIN [iot].[locations] l
ON m.locationID = l.id and l.nameID = @location
GROUP BY dateadd(mi, datediff(mi, 0, timestamp), 0)
ORDER BY timestamp DESC

In [ ]:
-- per hour
DECLARE @location VARCHAR(50);
SET @location = 'churchill-1c';

SELECT TOP 1000 dateadd(hour, datediff(hour, 0, timestamp), 0) as timestamp, AVG(temperatureDHT) as temperature, AVG(pressureLPS) as pressure, AVG(humidityDHT) as humidity, AVG(eco2) as eco2, AVG(wifiDevices) as wifiDevices, COUNT(timestamp) as '_count'
FROM [iot].[messages] m
INNER JOIN [iot].[locations] l
ON m.locationID = l.id and l.nameID = @location
GROUP BY dateadd(hour, datediff(hour, 0, timestamp), 0)
ORDER BY timestamp DESC

In [ ]:
-- per X minutes
DECLARE @location VARCHAR(50);
SET @location = 'cl-test-1';

DECLARE @perMinutes INT;
SET @perMinutes = 1;

SELECT TOP 1000 dateadd(mi, datediff(mi,0, timestamp) / @perMinutes * @perMinutes, 0) as timestamp, AVG(temperatureDHT) as temperature, AVG(pressureLPS) as pressure, AVG(humidityDHT) as humidity, AVG(eco2) as eco2, AVG(wifiDevices) as wifiDevices, COUNT(timestamp) as '_count'
FROM [iot].[messages] m
INNER JOIN [iot].[locations] l
ON m.locationID = l.id and l.nameID = @location
GROUP BY dateadd(mi, datediff(mi,0, timestamp) / @perMinutes * @perMinutes, 0)
ORDER BY timestamp DESC

In [ ]:
-- get the last data timestamp from each device
SELECT l.name as location, l.owner as devOwner, MAX(m.timestamp) as last_data
FROM [iot].[locations] l
LEFT JOIN [iot].[messages] m
ON l.id = m.locationID
GROUP BY l.name, l.owner

# Summary

In [ ]:
With JoinedData AS (
    SELECT l.name as name, l.nameID as locationID, dateadd(mi, datediff(mi,0, m.timestamp), 0) as timestamp, AVG(m.temperatureDHT) as temperature, AVG(m.pressureLPS) as pressure, AVG(m.humidityDHT) as humidity, AVG(m.eco2) as co2, AVG(m.wifiDevices) as devices, COUNT(m.timestamp) as '_count'
    FROM [iot].[messages] m
    INNER JOIN [iot].[locations] l
    ON m.locationID = l.id
    WHERE l.shown = 1
    GROUP BY l.name, l.nameID, datediff(mi,0, m.timestamp)
), WithIdx AS (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY name ORDER BY name ASC, timestamp DESC) as _idx
    FROM JoinedData
)
SELECT *
FROM WithIdx
WHERE _idx < 16